In [1]:
import pymatgen
import numpy as np
from mp_api.client import MPRester
import re
import pandas as pd
import torch


In [5]:

with MPRester("Lv3H4D1twE2kDiXZLWdaBTIRM82iRdCi") as mpr:
    docs = mpr.summary.search(
        chemsys="Nb-O", fields=["material_id", "formula_pretty"]
    )
    mpid_bgap_dict = {doc.material_id: doc.formula_pretty for doc in docs}


Retrieving SummaryDoc documents:   0%|          | 0/33 [00:00<?, ?it/s]

In [10]:
mpid_bgap_dict

{MPID(mp-1364247): 'Nb2O5',
 MPID(mp-510554): 'Nb12O29',
 MPID(mp-1470): 'Nb12O29',
 MPID(mp-2533): 'NbO2',
 MPID(mp-2692): 'NbO',
 MPID(mp-1095187): 'Nb2O3',
 MPID(mp-821): 'NbO2',
 MPID(mp-776975): 'NbO2',
 MPID(mp-1101660): 'Nb2O5',
 MPID(mp-755690): 'NbO2',
 MPID(mp-25214): 'NbO2',
 MPID(mp-1232391): 'Nb4O9',
 MPID(mp-1245189): 'NbO2',
 MPID(mp-1244973): 'NbO2',
 MPID(mp-649729): 'NbO2',
 MPID(mp-680944): 'Nb2O5',
 MPID(mp-634965): 'NbO',
 MPID(mp-556048): 'Nb2O5',
 MPID(mp-776896): 'Nb2O5',
 MPID(mp-1179977): 'NbO3',
 MPID(mp-581967): 'Nb2O5',
 MPID(mp-1186241): 'NbO3',
 MPID(mp-1595): 'Nb2O5',
 MPID(mp-557057): 'NbO2',
 MPID(mp-604): 'Nb2O5',
 MPID(mp-28212): 'Nb4O5',
 MPID(mp-1201852): 'Nb2O5',
 MPID(mp-10426): 'Nb2O5',
 MPID(mp-2311): 'NbO',
 MPID(mp-1220361): 'NbO3',
 MPID(mp-1244881): 'NbO2',
 MPID(mp-766361): 'Nb2O5',
 MPID(mp-754698): 'NbO2'}

In [40]:
### GETTING LIST OF ALL POSSIBLE OXIDATION STATES FOR ELEMENTS IN PERIODIC TABLE.
from bs4 import BeautifulSoup
import requests

In [53]:
url = 'https://en.wikipedia.org/wiki/Template:Infobox_element/symbol-to-oxidation-state'
response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find_all('table', {'class': 'wikitable'})  # Replace 'wikitable' with the class name or other identifying attributes of the target table


In [54]:
data = []
for row in table[1].find_all('tr'):
    row_data = []
    for cell in row.find_all(['td', 'th']):
        row_data.append(cell.text.strip())
    if row_data:
        data.append(row_data)

array_data = np.array(data)

C:\Users\songo\AppData\Local\Temp\ipykernel_13412\865070844.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  array_data = np.array(data)


In [95]:
import numpy as np
import re
def remove_enclosed_elements(elements):
    modified_elements = []

    for element in elements:
        # Remove elements surrounded by square brackets
        element = re.sub(r'\[[^]]*\]', '', element)

        # Remove elements surrounded by parentheses
        element = re.sub(r'\([^)]*\)', '', element)
        
        element = element.replace('+','')

        modified_elements.append(element)

    return modified_elements

# Example usage
modified_arr = remove_enclosed_elements(array_data[9])
print(modified_arr)




['7', 'nitrogen', 'N', '−3, −2, −1, 0, 1, 2, 3, 4, 5 ', '−3, 3, 5', '15', 'V', '-3, -2, -1, 1, 2, 3, 4, 5', '']


In [113]:
oxidation_states = ['Symbol', 'OxidationStates', 'MainStates']
for states in array_data[3:120]:
    cleanstate = remove_enclosed_elements(states)
    Symbol = cleanstate[2]
    oxstates = cleanstate[3]
    mainstates = cleanstate[4]
    oxidation_states = np.vstack([oxidation_states, [Symbol, oxstates, mainstates]])

In [121]:
# Checkpoint!
np.savetxt('oxidationstates.csv', oxidation_states, delimiter=',',fmt='%s')


In [125]:
import numpy as np

# Oxidation state data (replace with the actual data)
oxidation_states = oxidation_states[1:len(oxidation_states)-1]

# Function to parse oxidation states from the string
def parse_oxidation_states(oxidation_states_str):
    oxidation_states_str = oxidation_states_str.replace('−', '-')  # Replace minus sign character
    oxidation_states = oxidation_states_str.split(',')
    oxidation_states = [int(state.strip()) for state in oxidation_states if state.strip()]  # Strip whitespace characters
    return oxidation_states


# Function to generate oxide formulas based on charge balance
def generate_oxide_formulas():
    oxide_formulas = []
    for element_data in oxidation_states:
        element = element_data[0]
        element_states_str = element_data[1]
        element_states = parse_oxidation_states(element_states_str)
        for state in element_states:
            # Check if the element is oxygen
            if element == "O":
                continue  # Skip oxygen, as it cannot form an oxide with itself
            # Determine the required oxidation state of oxygen to satisfy charge balance
            oxygen_state = -sum(element_states) / len(element_states)
            # Check if the required oxidation state of oxygen is an integer
            if oxygen_state.is_integer() and oxygen_state != 0:
                # Generate the oxide formula based on the oxidation states
                oxide_formula = element + str(abs(state)) + "O" + str(int(abs(oxygen_state)))
                oxide_formulas.append(oxide_formula)
    return oxide_formulas

# Generate the oxide formulas
oxide_formulas = generate_oxide_formulas()

# Convert the list to a numpy array
oxide_formulas_array = np.array(oxide_formulas)

# Print the numpy array
print(oxide_formulas_array)


['Be0O1' 'Be1O1' 'Be2O1' 'N3O1' 'N2O1' 'N1O1' 'N0O1' 'N1O1' 'N2O1' 'N3O1'
 'N4O1' 'N5O1' 'Mg0O1' 'Mg1O1' 'Mg2O1' 'P3O1' 'P2O1' 'P1O1' 'P0O1' 'P1O1'
 'P2O1' 'P3O1' 'P4O1' 'P5O1' 'S2O2' 'S1O2' 'S0O2' 'S1O2' 'S2O2' 'S3O2'
 'S4O2' 'S5O2' 'S6O2' 'Ti2O1' 'Ti1O1' 'Ti0O1' 'Ti1O1' 'Ti2O1' 'Ti3O1'
 'Ti4O1' 'Mn3O2' 'Mn2O2' 'Mn1O2' 'Mn0O2' 'Mn1O2' 'Mn2O2' 'Mn3O2' 'Mn4O2'
 'Mn5O2' 'Mn6O2' 'Mn7O2' 'Ni2O1' 'Ni1O1' 'Ni0O1' 'Ni1O1' 'Ni2O1' 'Ni3O1'
 'Ni4O1' 'Ga5O1' 'Ga4O1' 'Ga3O1' 'Ga2O1' 'Ga1O1' 'Ga0O1' 'Ga1O1' 'Ga2O1'
 'Ga3O1' 'As3O1' 'As2O1' 'As1O1' 'As0O1' 'As1O1' 'As2O1' 'As3O1' 'As4O1'
 'As5O1' 'Se2O2' 'Se1O2' 'Se0O2' 'Se1O2' 'Se2O2' 'Se3O2' 'Se4O2' 'Se5O2'
 'Se6O2' 'Br1O3' 'Br1O3' 'Br2O3' 'Br3O3' 'Br4O3' 'Br5O3' 'Br7O3' 'Kr0O1'
 'Kr1O1' 'Kr2O1' 'Sb3O1' 'Sb2O1' 'Sb1O1' 'Sb0O1' 'Sb1O1' 'Sb2O1' 'Sb3O1'
 'Sb4O1' 'Sb5O1' 'Te2O2' 'Te1O2' 'Te0O2' 'Te1O2' 'Te2O2' 'Te3O2' 'Te4O2'
 'Te5O2' 'Te6O2' 'Xe0O4' 'Xe2O4' 'Xe4O4' 'Xe6O4' 'Xe8O4' 'Tb0O2' 'Tb1O2'
 'Tb2O2' 'Tb3O2' 'Tb4O2' 'Dy0O2' 'Dy1O2' 'Dy2O2' 'Dy3O